In [1]:
# pip install realtimetts[gttsengine] pyttsx3 
import requests
import json
from RealtimeTTS import TextToAudioStream, SystemEngine, GTTSEngine

In [2]:

def printSameLine(str):
    print(str, end="", flush=True)

def createMessage(content, isUser=True):
    return {
        'role': 'user' if isUser else 'assistant',
        'content': content
    }

def sendMessage(endpoint, model, history, content, callback = None):
    msg = createMessage(content)
    history.append(msg)

    req = requests.post(endpoint, json={
        'model': model,
        'messages': history,
        'stream': True,
    }, stream=True)

    if req.status_code != 200:
        raise Exception(f"Failed to send message: {req.status_code} {req.reason}")

    req.raise_for_status()

    total_msg = ""
    for l in req.iter_lines():
        res_json = json.loads(l)

        if "error" in res_json: raise Exception(res_json.get('error'))
        if res_json.get('done'): break

        res_msg = res_json.get('message').get('content')
        total_msg += res_msg

        if callback is not None: callback(res_msg)

    history.append(createMessage(total_msg, False))
    return total_msg

def makeTTSEngine(engine, stop_char_list = [".", "!", "?", ","]):
    engine = TextToAudioStream(engine)

    temp_sentence = ""
    def callback(token):
        nonlocal temp_sentence
        temp_sentence += token
        printSameLine(token)

        if any([c in token for c in stop_char_list]):
            engine.feed(temp_sentence)
            if not engine.is_playing():
                engine.play_async()
            temp_sentence = ""
            return

    return callback


In [3]:
model = 'llama3'
endpoint = 'http://localhost:11434/api/chat'
history = []
# tts_engine = makeTTSEngine(SystemEngine()) # offline (pyttsx3)
tts_engine = makeTTSEngine(GTTSEngine())
# can also use OpenAI voice service with their API

def sendAndPlay(msg):
    sendMessage(endpoint, model, history, msg, tts_engine)

pre_prompt_condition = [
    # 'your name is Nova',
    'completely verbal conversation',
    'short',
    'professional but casual',
]

sendAndPlay("Condition for this conversation: " + ", ".join(pre_prompt_condition))
print("\nReady...")

I'm ready to chat. What's on your mind?
Ready...


In [ ]:
# to stop current playing voice
# tts_engine.stop()

In [4]:
sendAndPlay("Can you tell me what is the best cake?")

That's a tough one! While opinions may vary, I think it's safe to say that a well-made classic vanilla or chocolate cake is hard to beat. But, if you're looking for something more unique, a good carrot cake with cream cheese frosting can't be ignored either! What's your go-to cake order?